In [1]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

ModuleNotFoundError: No module named 'langchain_groq'

In [ ]:
 pip install langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/")
page_data=loader.load().pop().page_content
print(page_data)

Search Jobs — Google CareersCareersSkip navigation linksTeamsLocationsBenefitsJobsStudentsYour careerarrow_drop_downYour career menu ApplicationsProfileSaved jobsJob alertsSign inCareersApplicationsProfileSearch jobsSaved jobsJob alertsTeamsLocationsBenefitsJobsStudentsHelpSend FeedbackSearch JobsSearch sidebar2,802  jobs matchedClear filtersWhat do you want to do?Locationsexpand_moreExperienceexpand_moreSkills & qualificationsexpand_moreDegreeexpand_moreJob typesexpand_moreOrganizationsexpand_moreSort byexpand_moreFollow Life at Google on More about usAbout usopen_in_newContact usopen_in_newPressopen_in_newRelated InformationInvestor relationsopen_in_newBlogopen_in_newEqual OpportunityGoogle is proud to be an equal opportunity and affirmative action employer. We are committed to building a workforce that is representative of the users we serve, creating a culture of belonging, and providing an equal employment opportunity regardless of race, creed, color, religion, gender, sexual orie

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt_extract=PromptTemplate.from_template(
    """
    ### SCRAPPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in json format containing the following keys:'roles','experience', 'skills',and 'description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract=prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)


```json
[
  {
    "roles": "Digital Business Marketing Apprenticeship, March 2025 Start (English)",
    "experience": "0-1 year",
    "skills": "Google Workspace, English",
    "description": "Bachelor's degree or equivalent practical experience. Maximum 1 year of experience in a Digital Business Marketing role post graduation. Experience with using Google Workspace (e.g., Gmail, Chrome, Docs, Sheets, etc.) or similar applications. Ability to communicate in English fluently, to understand and execute instructions and learn via training documentations and presentations."
  },
  {
    "roles": "YouTube Ads Viewer Experience Product Manager",
    "experience": "8-10 years",
    "skills": "Product management, Technical role",
    "description": "Bachelor's degree or equivalent practical experience. 8 years of experience in product management or related technical role. 3 years of experience taking technical products from conception to launch."
  },
  {
    "roles": "Customer Engineering Man

str

In [ ]:
type(res.content)

str

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res
type(json_res)

list

In [ ]:
type(json_res)

list

In [ ]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [ ]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [ ]:
links = collection.query(query_texts=job["skills"], n_results=2).get('metadatas', [])
print(links)

[[{'links': 'https://example.com/magento-portfolio'}, {'links': 'https://example.com/typescript-frontend-portfolio'}]]


In [ ]:

for job in json_res:
    print(job["skills"])  

Google Workspace, English
Product management, Technical role
Cloud native architecture, Leadership role
Electrical Engineering, Computer Engineering, Computer Science
Business Continuity, Resilience, Cybersecurity
Advertising, Sales, Marketing
Computer Science, Machine Learning, Artificial Intelligence
Computer Science, Kubernetes, Cloud native applications
Quantum Computing, Physics, Computer Science
Software development, Data structures, Algorithms
Electrical Engineering, Computer Engineering, Computer Science
Software development, Leadership role
Compliance, Risk management, Auditing
Program management, Technical project management
Mechanical/HVAC
HR, Operations, Process improvement
Mechanical Engineering, Finite Element Analysis
Software development, Data structures, Algorithms
Security management, Data center operations
Product management, Technical role


In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Swaathy Balaji, a business development executive at IntelliFusion. Intellifusion is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Intellifusion
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase IntelliFusion's portfolio: {link_list}
        Remember you are Swaathy, BDE at IntelliFusion. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Product Management Solutions for Meet Hardware, Google Cloud

Dear Hiring Manager,

I came across the job description for a Product Manager, Meet Hardware, Google Cloud, and I'm excited to introduce IntelliFusion as a potential partner to help you find the ideal candidate or provide expert product management solutions to drive your business forward.

At IntelliFusion, we specialize in AI & Software Consulting, empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts has a proven track record of delivering high-quality solutions that meet the unique needs of our clients.

With our expertise in product management and technical roles, we can help you find a skilled Product Manager who can take your technical products from conception to launch. Our team has experience working with video conferencing products, and we're confident that we can provide the right candidate